In [1]:
!pip install tensorflow==2.3.0
#Gym is a standard API for reinforcement learning, 
#and a diverse collection of reference environments
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame

In [1]:
#Key dependencies for openAI gym
import gym
import random

In [2]:
#Environment setup-making cartpole environment
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
#getting actions, here it is 2 i.e left and right
actions = env.action_space.n
actions


C:\Users\Asus\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\Asus\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


2

## Building a random environment, just randomly playing game to see results

In [24]:

episodes = 10
for episode in range(1,episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    #while not done is same as while (done==False)
    #run the loop while it is equal to false and exit when equal to True,(while not (opposite of done) run)
    while not done:
        env.render()
        action = random.choice([0,1])
        n_state , reward , done , info = env.step(action)
        #if we take step in correct direction and we have not failed, we get 1 point(i.e reward)
        #if we failed or we get at the end of the game, "done" will be set to true.
        #
        score += reward
    print('Episode:{} Score{}'.format(episode, score))
    


Episode:1 Score24.0
Episode:2 Score15.0
Episode:3 Score15.0
Episode:4 Score37.0
Episode:5 Score20.0
Episode:6 Score13.0
Episode:7 Score36.0
Episode:8 Score26.0
Episode:9 Score11.0
Episode:10 Score41.0


## importing Tensorflow keras dependencies


In [30]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
#error occured here(no module named termcolor), solved by deleting package from c:Users:Asus:anaconda3->Lib-> site packages and reinstalled
#using !pip install termcolor


In [25]:
#building sequential model
def build_model(states,actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation = 'relu'))
    model.add(Dense(24,activation = 'relu'))
    model.add(Dense(actions,activation = 'linear'))
    return model
#help(Dense)

In [36]:
#model = build_model(states, actions)

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## Importing keras Reinforcement Learning(RL) dependencies and building the model

In [36]:

#DeepQ network agent, various agents are available in keras RL like DQN,NAPA,DDPG,SARSA,CEM agents
from rl.agents import DQNAgent
#we are using Policy-based RL
from rl.policy import BoltzmannQPolicy
#we need to maintain some memory for our DQN agent,SequentialMemory class allows us to do that
from rl.memory import SequentialMemory

In [37]:

model = build_model(states, actions)

In [38]:
#wrapping inside a function so we can reproduce it when we want to reload it from memory
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 50000 , window_length =1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy,
                   nb_actions = actions, nb_steps_warmup = 10, target_model_update = 1e-2)
    return dqn
    

In [39]:

dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 1)
#help(Adam)
#error occured Keras-rl2 error AttributeError: 'Sequential' object has no attribute '_compile_time_distribution_strategy'
#solved by reversing the order of the cells i.e model = build_model cell after keras RL dependencies cell

C:\Users\Asus\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training_utils.py:1446: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if isinstance(sample_weight_mode, collections.Mapping):


Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 17:40 - reward: 1.0000

C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 15 + 1) inste

   29/10000 [..............................] - ETA: 7:00 - reward: 1.0000

C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 19 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 20 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 21 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 22 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
C:\Users\Asus\anaconda3\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 23 + 1) inste

10000/10000 [==============================] - 115s 12ms/step - reward: 1.0000
88 episodes - episode_reward: 111.295 [10.000, 450.000] - loss: 2.468 - mae: 19.228 - mean_q: 38.933

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 116s 12ms/step - reward: 1.0000
42 episodes - episode_reward: 238.357 [171.000, 343.000] - loss: 3.367 - mae: 41.133 - mean_q: 83.257

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 117s 12ms/step - reward: 1.0000
39 episodes - episode_reward: 259.410 [191.000, 449.000] - loss: 2.512 - mae: 45.061 - mean_q: 90.956

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 119s 12ms/step - reward: 1.0000
43 episodes - episode_reward: 232.512 [164.000, 465.000] - loss: 1.632 - mae: 43.360 - mean_q: 87.330

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 121s 12ms/step - reward: 1.0000
done, took 588.401 seconds


In [40]:
scores = dqn.test(env, nb_episodes = 100 ,visualize = False)
print(np.mean(scores.history['episode_reward']))
#print(scores.history)

Testing for 100 episodes ...
Episode 1: reward: 272.000, steps: 272
Episode 2: reward: 263.000, steps: 263
Episode 3: reward: 203.000, steps: 203
Episode 4: reward: 268.000, steps: 268
Episode 5: reward: 289.000, steps: 289
Episode 6: reward: 267.000, steps: 267
Episode 7: reward: 281.000, steps: 281
Episode 8: reward: 170.000, steps: 170
Episode 9: reward: 273.000, steps: 273
Episode 10: reward: 186.000, steps: 186
Episode 11: reward: 244.000, steps: 244
Episode 12: reward: 180.000, steps: 180
Episode 13: reward: 186.000, steps: 186
Episode 14: reward: 301.000, steps: 301
Episode 15: reward: 215.000, steps: 215
Episode 16: reward: 208.000, steps: 208
Episode 17: reward: 279.000, steps: 279
Episode 18: reward: 286.000, steps: 286
Episode 19: reward: 274.000, steps: 274
Episode 20: reward: 234.000, steps: 234
Episode 21: reward: 239.000, steps: 239
Episode 22: reward: 209.000, steps: 209
Episode 23: reward: 191.000, steps: 191
Episode 24: reward: 241.000, steps: 241
Episode 25: reward: 

In [41]:
scores = dqn.test(env, nb_episodes = 5 ,visualize = True)

Testing for 5 episodes ...
Episode 1: reward: 187.000, steps: 187
Episode 2: reward: 257.000, steps: 257
Episode 3: reward: 301.000, steps: 301
Episode 4: reward: 190.000, steps: 190
Episode 5: reward: 237.000, steps: 237


In [42]:
#steps to save the model and use it later on eg for production,
#i.e we will save our weights and then reload them when we need
dqn.save_weights('dqn_weights.h5f', overwrite =True)


In [ ]:
#help(dqn.test)

## Deleting the model and reloading it with saved weights

In [44]:
del model
del dqn
del env

In [47]:
env = gym.make('CartPole-v1')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics = ['mae'])

In [48]:
dqn.load_weights("dqn_weights.h5f")

In [49]:
scores = dqn.test(env, nb_episodes = 5, visualize = True)

Testing for 5 episodes ...
Episode 1: reward: 209.000, steps: 209
Episode 2: reward: 217.000, steps: 217
Episode 3: reward: 194.000, steps: 194
Episode 4: reward: 345.000, steps: 345
Episode 5: reward: 284.000, steps: 284
